In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
path = os.path.join('data', 'US', 'us_data_weekly.csv')
dus = pd.read_csv(path, index_col=0)

In [3]:
for lag in [1,2,5,10,15,20,25,30,40,50]:
    dus[f'DGS1MO_t-{lag}'] = dus['DGS1MO'].shift(lag-1)
    dus[f'DGS3MO_t-{lag}'] = dus['DGS3MO'].shift(lag-1)
    dus[f'DGS6MO_t-{lag}'] = dus['DGS6MO'].shift(lag-1)
    dus[f'DGS1_t-{lag}'] = dus['DGS1'].shift(lag-1)
  

for lag in [1,2,10]:
    dus[f'DGS1_t-{lag}'] = dus['DGS1'].shift(lag-1)
    dus[f'DGS2_t-{lag}'] = dus['DGS2'].shift(lag-1)
    dus[f'DGS3_t-{lag}'] = dus['DGS3'].shift(lag-1)
    dus[f'DGS5_t-{lag}'] = dus['DGS5'].shift(lag-1)
    dus[f'DGS7_t-{lag}'] = dus['DGS7'].shift(lag-1)
    dus[f'DGS10_t-{lag}'] = dus['DGS10'].shift(lag-1)
    dus[f'DGS20_t-{lag}'] = dus['DGS20'].shift(lag-1)
    dus[f'DGS30_t-{lag}'] = dus['DGS30'].shift(lag-1)

In [4]:
dus['Y_1MO'] = dus['DGS1MO'].shift(-1)
dus['Y_3MO'] = dus['DGS3MO'].shift(-1)
dus['Y_6MO'] = dus['DGS6MO'].shift(-1)
dus['Y_1year'] = dus['DGS1'].shift(-1)
dus['Y_2year'] = dus['DGS2'].shift(-1)
dus['Y_3year'] = dus['DGS3'].shift(-1)
dus['Y_5year'] = dus['DGS5'].shift(-1)
dus['Y_7year'] = dus['DGS7'].shift(-1)
dus['Y_10year'] = dus['DGS10'].shift(-1)
dus['Y_20year'] = dus['DGS20'].shift(-1)
dus['Y_30year'] = dus['DGS30'].shift(-1)

In [5]:
def add_ts_features(df, cols, max_lag=30, windows=[20, 60]):
    features = pd.DataFrame(index=df.index)
    
    for col in cols:
        y = df[col]

  
        # --- Statistiques glissantes ---
        for w in windows:
            features[f'{col}_mean_{w}'] = y.rolling(w).mean()
            features[f'{col}_std_{w}'] = y.rolling(w).std()
            features[f'{col}_q25_{w}'] = y.rolling(w).quantile(0.25)
            features[f'{col}_q75_{w}'] = y.rolling(w).quantile(0.75)
            features[f'{col}_q05_{w}'] = y.rolling(w).quantile(0.1)
            features[f'{col}_q90_{w}'] = y.rolling(w).quantile(0.9)
            features[f'{col}_range_{w}'] = y.rolling(w).max() - y.rolling(w).min() 
        
       
        # --- Autocorrélations locales ---
        for lag in range(1, max_lag + 1):
            features[f'{col}_autocorr_{lag}'] = (
                y.rolling(window=max(windows)).apply(lambda x: x.autocorr(lag=lag), raw=False)
            )
        
    return features

In [6]:
# Exemple d’usage :
cols = ['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3',
        'DGS5', 'DGS7', 'DGS10', 'DGS20', 'DGS30']

dus_features = add_ts_features(dus, cols)
dus_features.dropna()

/var/folders/l4/rqys14ws2955nprxbdsj722r0000gn/T/ipykernel_37268/2986023610.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[f'{col}_q90_{w}'] = y.rolling(w).quantile(0.9)
/var/folders/l4/rqys14ws2955nprxbdsj722r0000gn/T/ipykernel_37268/2986023610.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[f'{col}_range_{w}'] = y.rolling(w).max() - y.rolling(w).min()
/var/folders/l4/rqys14ws2955nprxbdsj722r0000gn/T/ipykernel_37268/2986023610.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usual

,DGS1MO_mean_20,DGS1MO_std_20,DGS1MO_q25_20,DGS1MO_q75_20,DGS1MO_q05_20,DGS1MO_q90_20,DGS1MO_range_20,DGS1MO_mean_60,DGS1MO_std_60,DGS1MO_q25_60,...,DGS30_autocorr_21,DGS30_autocorr_22,DGS30_autocorr_23,DGS30_autocorr_24,DGS30_autocorr_25,DGS30_autocorr_26,DGS30_autocorr_27,DGS30_autocorr_28,DGS30_autocorr_29,DGS30_autocorr_30
2004-02-27,0.0030,0.049001,-0.0150,0.0300,-0.052,0.042,0.23,-0.003833,0.054309,-0.0300,...,-0.259523,-0.077259,0.159453,-0.306692,-0.080479,0.071351,-0.010549,-0.041780,-0.080879,0.245651
2004-03-05,0.0025,0.048976,-0.0150,0.0300,-0.052,0.042,0.23,-0.003667,0.054305,-0.0300,...,-0.280439,-0.077388,0.157691,-0.292699,-0.092806,0.066148,-0.007043,-0.032955,-0.136969,0.266099
2004-03-12,0.0020,0.048839,-0.0150,0.0300,-0.052,0.042,0.23,-0.003500,0.054332,-0.0300,...,-0.350001,-0.033358,0.181366,-0.272465,-0.085109,0.104542,0.049654,0.013500,-0.149527,0.341943
2004-03-19,-0.0015,0.048480,-0.0300,0.0225,-0.052,0.042,0.23,-0.003500,0.054332,-0.0300,...,-0.334521,-0.013463,0.199492,-0.269040,-0.063045,0.152307,0.088528,0.003310,-0.122728,0.304756
2004-03-26,-0.0005,0.048501,-0.0300,0.0225,-0.052,0.042,0.23,-0.003833,0.054184,-0.0300,...,-0.334980,-0.012323,0.200922,-0.262623,-0.048590,0.161128,0.072734,0.017370,-0.139026,0.289036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-06,-0.0455,0.092365,-0.1075,0.0125,-0.145,0.041,0.39,-0.017167,0.061261,-0.0325,...,0.139276,-0.231618,-0.123862,0.206619,0.008628,0.407258,0.122133,-0.091456,0.147711,-0.167988
2024-12-13,-0.0530,0.093702,-0.1300,0.0050,-0.145,0.041,0.39,-0.018833,0.063221,-0.0325,...,0.085310,-0.243816,-0.109734,0.163261,-0.001877,0.231416,0.030619,0.030235,0.172196,-0.160077
2024-12-20,-0.0555,0.091449,-0.1300,0.0000,-0.145,0.031,0.39,-0.019000,0.063157,-0.0325,...,0.084048,-0.232134,-0.108906,0.163118,0.034730,0.248380,-0.020838,0.009172,0.183212,-0.152127
2024-12-27,-0.0550,0.091795,-0.1300,0.0025,-0.145,0.031,0.39,-0.018167,0.063205,-0.0300,...,0.007539,-0.160140,-0.070777,0.211115,0.059892,0.227114,-0.020601,-0.071118,0.176823,-0.102448


In [7]:
dus = dus.merge(dus_features, how = 'inner', left_index=True, right_index=True)
dus = dus.dropna()
dus

,RPI,INDPRO,UNRATE,UEMP5TO14,PAYEMS,USGOOD,USCONS,MANEMP,DMANEMP,NDMANEMP,...,DGS30_autocorr_21,DGS30_autocorr_22,DGS30_autocorr_23,DGS30_autocorr_24,DGS30_autocorr_25,DGS30_autocorr_26,DGS30_autocorr_27,DGS30_autocorr_28,DGS30_autocorr_29,DGS30_autocorr_30
2004-02-27,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.259523,-0.077259,0.159453,-0.306692,-0.080479,0.071351,-0.010549,-0.041780,-0.080879,0.245651
2004-03-05,0.003610,-0.003912,0.035091,-0.000821,0.002435,0.002992,0.007140,0.000560,0.000902,0.000000,...,-0.280439,-0.077388,0.157691,-0.292699,-0.092806,0.066148,-0.007043,-0.032955,-0.136969,0.266099
2004-03-12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.350001,-0.033358,0.181366,-0.272465,-0.085109,0.104542,0.049654,0.013500,-0.149527,0.341943
2004-03-19,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.334521,-0.013463,0.199492,-0.269040,-0.063045,0.152307,0.088528,0.003310,-0.122728,0.304756
2004-03-26,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.334980,-0.012323,0.200922,-0.262623,-0.048590,0.161128,0.072734,0.017370,-0.139026,0.289036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.152791,-0.267233,-0.108337,0.218529,-0.054965,0.373189,0.083250,-0.111179,0.176063,-0.161267
2024-12-06,0.001065,0.010677,-0.024098,-0.032515,0.002034,0.000185,0.001811,-0.000783,-0.001895,0.001031,...,0.139276,-0.231618,-0.123862,0.206619,0.008628,0.407258,0.122133,-0.091456,0.147711,-0.167988
2024-12-13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.085310,-0.243816,-0.109734,0.163261,-0.001877,0.231416,0.030619,0.030235,0.172196,-0.160077
2024-12-20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.084048,-0.232134,-0.108906,0.163118,0.034730,0.248380,-0.020838,0.009172,0.183212,-0.152127


In [8]:
dus = dus.drop(columns=['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3','DGS5', 'DGS7', 'DGS10', 'DGS20', 'DGS30'])

In [9]:
Y = dus[['Y_1MO', 'Y_3MO', 'Y_6MO', 'Y_1year', 'Y_2year', 'Y_3year', 'Y_5year', 'Y_7year', 'Y_10year', 'Y_20year', 'Y_30year']]

corrs = pd.DataFrame({
    target: dus.corrwith(Y[target]) for target in Y.columns
}).abs()  

# repérer les colonnes où la corrélation absolue < 0.05 pour toutes les targets
mask = (corrs < 0.1).all(axis=1)
low_corr_features = corrs.index[mask]

# supprimer ces colonnes
dus_filtered = dus.drop(columns=low_corr_features)

print(f"{len(low_corr_features)} features were deleted")

436 features were deleted


In [10]:
print(dus_filtered.shape)

(1088, 187)


In [11]:
# Required libs
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

In [12]:
# 1. CONFIG
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

SEQ_LEN = 12          # input sequence length in weeks
LSTM_UNITS = 64
DROPOUT = 0.2
BATCH_SIZE = 32
EPOCHS = 200

In [13]:
# 2. LOAD DATA (placeholder)
target_cols = ['Y_1MO', 'Y_3MO', 'Y_6MO', 'Y_1year', 'Y_2year', 'Y_3year', 'Y_5year', 'Y_7year', 'Y_10year', 'Y_20year', 'Y_30year']

# Example: df = pd.read_csv('weekly_macro_yields.csv', parse_dates=['date'], index_col='date')
# Ensure df sorted by date ascending
dus_filtered = dus_filtered.sort_index()

In [14]:
# 3. PREPROCESS: scaling and sequences

df = dus_filtered

# split train / val / test by time (no leakage)
n_obs = len(df)
train_frac = 0.7
val_frac = 0.15
train_end = int(n_obs * train_frac)
val_end = int(n_obs * (train_frac + val_frac))

df_train = df.iloc[:train_end]
df_val = df.iloc[train_end:val_end]
df_test = df.iloc[val_end:]

# Fit scalers on training data only
feature_cols = [c for c in df.columns if c not in target_cols]
X_scaler = StandardScaler().fit(df_train[feature_cols])
y_scaler = StandardScaler().fit(df_train[target_cols])  # scale targets too

def create_sequences(data_df, seq_len=SEQ_LEN):
    Xs, ys = [], []
    data_vals = data_df[feature_cols].values
    target_vals = data_df[target_cols].values
    n = len(data_df)
    for i in range(n - seq_len):
        X_seq = data_vals[i : i + seq_len]
        y_target = target_vals[i + seq_len]   # single-step next week
        Xs.append(X_seq)
        ys.append(y_target)
    return np.array(Xs), np.array(ys)

# Scale dataframes
df_scaled = df.copy()
df_scaled[feature_cols] = X_scaler.transform(df[feature_cols])
df_scaled[target_cols] = y_scaler.transform(df[target_cols])

In [15]:
# Create sequences using scaled df and then split by indices consistently
X_all, y_all = create_sequences(df_scaled, SEQ_LEN)

In [16]:
# Determine sample counts matching earlier splits
# First index in sequences corresponds to original row index seq_len
idx_seq_start = df_scaled.index[SEQ_LEN]
# Compute counts for train/val/test
n_total = len(df_scaled)
train_count = max(0, train_end - SEQ_LEN)
val_count = max(0, val_end - SEQ_LEN) - train_count
test_count = len(X_all) - train_count - val_count

In [17]:
df_train.shape

(761, 187)

In [18]:
X_train = X_all[:train_count]
y_train = y_all[:train_count]
X_val = X_all[train_count:train_count+val_count]
y_val = y_all[train_count:train_count+val_count]
X_test = X_all[train_count+val_count:]
y_test = y_all[train_count+val_count:]

print("Shapes:", X_train.shape, y_train.shape, X_val.shape, X_test.shape)

Shapes: (749, 12, 176) (749, 11) (163, 12, 176) (164, 12, 176)


In [20]:
# 4. MODEL (single LSTM, multi-output)
n_features = X_train.shape[2]
n_targets = y_train.shape[1]

inp = Input(shape=(SEQ_LEN, n_features))
x = LSTM(LSTM_UNITS, return_sequences=False)(inp)
x = Dropout(DROPOUT)(x)
# optional dense hidden layer
x = Dense(64, activation='relu')(x)
out = Dense(n_targets, activation='linear')(x)

model = Model(inputs=inp, outputs=out)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 12, 176)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        61,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 11)             │           715 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 66,571 (260.04 KB)

 Trainable params: 66,571 (260.04 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# 5. CALLBACKS
es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
chk = ModelCheckpoint('best_lstm.h5', monitor='val_loss', save_best_only=True, verbose=1)

In [ ]:
# 6. TRAIN
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[es, chk],
    verbose=2,
    shuffle=False  # keep order; optional to set True for mini-batch randomness
)

Epoch 1/200

Epoch 1: val_loss improved from None to 0.84996, saving model to best_lstm.h5


24/24 - 1s - 28ms/step - loss: 1.0886 - mae: 0.7185 - val_loss: 0.8500 - val_mae: 0.5885
Epoch 2/200

Epoch 2: val_loss improved from 0.84996 to 0.83105, saving model to best_lstm.h5


24/24 - 0s - 5ms/step - loss: 0.9870 - mae: 0.6729 - val_loss: 0.8310 - val_mae: 0.5754
Epoch 3/200

Epoch 3: val_loss did not improve from 0.83105
24/24 - 0s - 4ms/step - loss: 0.9538 - mae: 0.6589 - val_loss: 0.8326 - val_mae: 0.5753
Epoch 4/200

Epoch 4: val_loss improved from 0.83105 to 0.83001, saving model to best_lstm.h5


24/24 - 0s - 4ms/step - loss: 0.9270 - mae: 0.6495 - val_loss: 0.8300 - val_mae: 0.5736
Epoch 5/200

Epoch 5: val_loss did not improve from 0.83001
24/24 - 0s - 4ms/step - loss: 0.9011 - mae: 0.6415 - val_loss: 0.8484 - val_mae: 0.5853
Epoch 6/200

Epoch 6: val_loss did not improve from 0.83001
24/24 - 0s - 4ms/step - loss: 0.8842 - mae: 0.6377 - val_loss: 0.8346 - val_mae: 0.5753
Epoch 7/200

Epoch 7: val_loss did not improve from 0.83001
24/24 - 0s - 4ms/step - loss: 0.8743 - mae: 0.6308 - val_loss: 0.8746 - val_mae: 0.5935
Epoch 8/200

Epoch 8: val_loss did not improve from 0.83001
24/24 - 0s - 4ms/step - loss: 0.8850 - mae: 0.6356 - val_loss: 0.8554 - val_mae: 0.5840
Epoch 9/200

Epoch 9: val_loss did not improve from 0.83001
24/24 - 0s - 4ms/step - loss: 0.8994 - mae: 0.6441 - val_loss: 0.8693 - val_mae: 0.5901
Epoch 10/200

Epoch 10: val_loss improved from 0.83001 to 0.82934, saving model to best_lstm.h5


24/24 - 0s - 4ms/step - loss: 0.8882 - mae: 0.6401 - val_loss: 0.8293 - val_mae: 0.5713
Epoch 11/200

Epoch 11: val_loss did not improve from 0.82934
24/24 - 0s - 4ms/step - loss: 0.8193 - mae: 0.6215 - val_loss: 0.8603 - val_mae: 0.5913
Epoch 12/200

Epoch 12: val_loss did not improve from 0.82934
24/24 - 0s - 4ms/step - loss: 0.7848 - mae: 0.6129 - val_loss: 0.8814 - val_mae: 0.5965
Epoch 13/200

Epoch 13: val_loss did not improve from 0.82934
24/24 - 0s - 4ms/step - loss: 0.7532 - mae: 0.5997 - val_loss: 0.8733 - val_mae: 0.5938
Epoch 14/200

Epoch 14: val_loss did not improve from 0.82934
24/24 - 0s - 4ms/step - loss: 0.7315 - mae: 0.5903 - val_loss: 0.9112 - val_mae: 0.6105
Epoch 15/200

Epoch 15: val_loss did not improve from 0.82934
24/24 - 0s - 4ms/step - loss: 0.7198 - mae: 0.5856 - val_loss: 0.9516 - val_mae: 0.6271
Epoch 16/200

Epoch 16: val_loss did not improve from 0.82934
24/24 - 0s - 4ms/step - loss: 0.7691 - mae: 0.5979 - val_loss: 0.8871 - val_mae: 0.6083
Epoch 17/200

In [20]:
# 7. PREDICT & INVERSE TRANSFORM
y_pred_scaled = model.predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred_scaled)
y_true = y_scaler.inverse_transform(y_test)

# Evaluate RMSE per maturity
from sklearn.metrics import mean_squared_error
for i, col in enumerate(target_cols):
    rmse = np.sqrt(mean_squared_error(y_true[:, i], y_pred[:, i]))
    mae = np.mean(np.abs(y_true[:, i] - y_pred[:, i]))
    print(f"{col}: RMSE = {rmse:.4f}, MAE = {mae:.4f}")

# Save model if desired
model.save('lstm_multi_yield.h5')

NameError: name 'model' is not defined

In [21]:
def hit_ratio(y_true, y_pred):
    """
    Compute hit ratio (directional accuracy) for multi-output regression.
    
    Returns:
        hit_rates: list of hit ratios per target (in %)
        overall:   average hit ratio across all targets
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Compute actual and predicted changes (differences)
    true_change = np.diff(y_true, axis=0)   # shape: (n-1, n_targets)
    pred_change = np.diff(y_pred, axis=0)   # shape: (n-1, n_targets)
    
    mask = (true_change != 0) & (pred_change != 0)  # only count real moves
    correct = (np.sign(true_change) == np.sign(pred_change)) & mask
    hit_rates = np.mean(correct, axis=0, where=mask) * 100
    
    # Overall hit ratio (average across targets)
    overall = np.mean(hit_rates)
    
    return hit_rates.tolist(), float(overall)

In [61]:
per_target, overall = hit_ratio(y_true, y_pred)

print(f"Hit ratio per target: {per_target}")
print(f"Overall hit ratio: {overall:.2f}%")

Hit ratio per target: [48.734177215189874, 45.22292993630573, 49.693251533742334, 53.70370370370371, 53.70370370370371, 51.533742331288344, 45.06172839506173, 45.39877300613497, 46.012269938650306, 50.314465408805034, 53.70370370370371]
Overall hit ratio: 49.37%


In [22]:
# 1. Compute changes
y_diff = np.diff(y_all, axis=0)                  # shape: (n_samples-1, n_targets)

# 2. Convert to direction: 1 = up, 0 = down
#    (optional: exclude flat moves with a threshold)
THRESHOLD = 0.0  # set to 0.01 to ignore tiny moves
y_dir = (y_diff > THRESHOLD).astype(int)
y_train = y_dir[:train_count]
y_val = y_dir[train_count:train_count+val_count]
y_test = y_dir[train_count+val_count:]

In [23]:
n_features = X_train.shape[2]
n_targets = y_train.shape[1]

inp = Input(shape=(SEQ_LEN, n_features))
x = LSTM(LSTM_UNITS, return_sequences=False)(inp)
x = Dropout(DROPOUT)(x)
x = Dense(32, activation='relu')(x)

# One output per target → probability of "UP"
out = Dense(n_targets, activation='sigmoid')(x)  # [0,1] per target

LSTM_cls = Model(inp, out)
LSTM_cls.compile(
    optimizer='adam',
    loss='binary_crossentropy',      # perfect for 0/1 labels
    metrics=['accuracy']
)
LSTM_cls.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 12, 176)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        61,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 11)             │           363 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,139 (250.54 KB)

 Trainable params: 64,139 (250.54 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# 6. TRAIN
history = LSTM_cls.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=1,
    batch_size=BATCH_SIZE,
    verbose=2,
    callbacks=[es, chk],
    shuffle=False  # keep order; optional to set True for mini-batch randomness
)

: 

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

In [ ]:
# 3. PREPROCESS: scaling and sequences

df = dus_filtered

# split train / val / test by time (no leakage)
n_obs = len(df)
train_frac = 0.7
val_frac = 0.15
train_end = int(n_obs * train_frac)
val_end = int(n_obs * (train_frac + val_frac))

df_train = df.iloc[:train_end]
df_val = df.iloc[train_end:val_end]
df_test = df.iloc[val_end:]

# Fit scalers on training data only
feature_cols = [c for c in df.columns if c not in target_cols]
X_scaler = StandardScaler().fit(df_train[feature_cols])
y_scaler = StandardScaler().fit(df_train[target_cols])  # scale targets too

def create_sequences(data_df, seq_len=SEQ_LEN):
    Xs, ys = [], []
    data_vals = data_df[feature_cols].values
    target_vals = data_df[target_cols].values
    n = len(data_df)
    for i in range(n - seq_len):
        X_seq = data_vals[i : i + seq_len]
        y_target = target_vals[i + seq_len]   # single-step next week
        Xs.append(X_seq)
        ys.append(y_target)
    return np.array(Xs), np.array(ys)

# Scale dataframes
df_scaled = df.copy()
df_scaled[feature_cols] = X_scaler.transform(df[feature_cols])
df_scaled[target_cols] = y_scaler.transform(df[target_cols])

In [28]:
# 2. LOAD DATA (placeholder)
df = dus_filtered.copy()
df = df.sort_index()
target_cols = ['Y_1MO', 'Y_3MO', 'Y_6MO', 'Y_1year', 'Y_2year', 'Y_3year', 'Y_5year', 'Y_7year', 'Y_10year', 'Y_20year', 'Y_30year']

# Ensure df sorted by date ascending
feature_cols = df.columns.difference(target_cols)
directions = (df[target_cols].shift(-1) > df[target_cols]).astype(int)[:-1]
features = df[feature_cols][:-1]

# split train / val / test by time (no leakage)
n_obs = len(df)
train_frac = 0.7
val_frac = 0.15
train_end = int(n_obs * train_frac)
val_end = int(n_obs * (train_frac + val_frac))

features_train = features.iloc[:train_end]
features_scaler = StandardScaler().fit(features_train)
features_scaled = features_scaler.fit_transform(features)

X = features_scaled
y = directions.values

In [40]:
# --- Parameters ---
SEQ_LEN = 10
BATCH_SIZE = 64
HIDDEN_DIM = 64
LR = 1e-3
EPOCHS = 200

In [ ]:
# 2. Create PyTorch dataset
class YieldDataset(Dataset):
    def __init__(self, X, y, seq_length=SEQ_LEN):
        self.X = X
        self.y = y
        self.seq_length = seq_length
        
    def __len__(self):
        return len(self.X) - self.seq_length
    
    def __getitem__(self, idx):
        x_seq = self.X[idx:idx+self.seq_length]
        y_seq = self.y[idx+self.seq_length]
        return torch.tensor(x_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)

In [66]:
train_end/52

14.615384615384615

In [34]:
n_obs = len(X)
train_end = int(n_obs * 0.7)
val_end = int(n_obs * 0.85)

train_dataset = YieldDataset(X[:train_end], y[:train_end], SEQ_LEN)
val_dataset   = YieldDataset(X[train_end:val_end], y[train_end:val_end], SEQ_LEN)
test_dataset  = YieldDataset(X[val_end:], y[val_end:], SEQ_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [36]:
# Define LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :]  # take last time step
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

In [37]:
input_dim = X.shape[1]
output_dim = y.shape[1]
model = LSTMClassifier(input_dim, HIDDEN_DIM, output_dim)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [41]:
# Training loop with validation
for epoch in range(EPOCHS):
    model.train()
    train_losses = []
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {np.mean(train_losses):.4f} | Val Loss: {np.mean(val_losses):.4f}")

Epoch 1/200 | Train Loss: 0.6113 | Val Loss: 0.7069
Epoch 2/200 | Train Loss: 0.6029 | Val Loss: 0.7070
Epoch 3/200 | Train Loss: 0.5922 | Val Loss: 0.7108
Epoch 4/200 | Train Loss: 0.5816 | Val Loss: 0.7129
Epoch 5/200 | Train Loss: 0.5702 | Val Loss: 0.7171
Epoch 6/200 | Train Loss: 0.5580 | Val Loss: 0.7181
Epoch 7/200 | Train Loss: 0.5469 | Val Loss: 0.7210
Epoch 8/200 | Train Loss: 0.5350 | Val Loss: 0.7257
Epoch 9/200 | Train Loss: 0.5237 | Val Loss: 0.7272
Epoch 10/200 | Train Loss: 0.5107 | Val Loss: 0.7320
Epoch 11/200 | Train Loss: 0.4985 | Val Loss: 0.7332
Epoch 12/200 | Train Loss: 0.4831 | Val Loss: 0.7343
Epoch 13/200 | Train Loss: 0.4711 | Val Loss: 0.7389
Epoch 14/200 | Train Loss: 0.4581 | Val Loss: 0.7407
Epoch 15/200 | Train Loss: 0.4458 | Val Loss: 0.7464
Epoch 16/200 | Train Loss: 0.4336 | Val Loss: 0.7492
Epoch 17/200 | Train Loss: 0.4218 | Val Loss: 0.7505
Epoch 18/200 | Train Loss: 0.4112 | Val Loss: 0.7564
Epoch 19/200 | Train Loss: 0.3976 | Val Loss: 0.7629
Ep

In [39]:
# Evaluation on test set
model.eval()
y_true_list, y_pred_list = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        preds = (outputs > 0.5).float()
        y_true_list.append(y_batch.numpy())
        y_pred_list.append(preds.numpy())

y_true = np.concatenate(y_true_list)
y_pred = np.concatenate(y_pred_list)

acc_per_yield = (y_true == y_pred).mean(axis=0)
overall_acc = (y_true == y_pred).mean()

print("\nAccuracy per yield:")
for name, acc in zip(target_cols, acc_per_yield):
    print(f"{name}: {acc:.3f}")
print(f"Overall accuracy: {overall_acc:.3f}")


Accuracy per yield:
Y_1MO: 0.455
Y_3MO: 0.506
Y_6MO: 0.494
Y_1year: 0.519
Y_2year: 0.500
Y_3year: 0.474
Y_5year: 0.494
Y_7year: 0.442
Y_10year: 0.468
Y_20year: 0.455
Y_30year: 0.474
Overall accuracy: 0.480


In [58]:
HIDDEN_DIM = 16   # smaller
LR = 5e-4
BATCH_SIZE = 32
EPOCHS = 50
dropout = 0.5

In [59]:
class RLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

In [60]:
input_dim = X.shape[1]
output_dim = y.shape[1]
model = RLSTMClassifier(input_dim, HIDDEN_DIM, output_dim)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-4)

In [61]:
best_val_loss = float('inf')
patience, patience_counter = 5, 0

for epoch in range(EPOCHS):
    model.train()
    train_losses = []
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    model.eval()
    val_losses = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())
    val_loss = np.mean(val_losses)
    print(f"Epoch {epoch+1}/{EPOCHS} | Train {np.mean(train_losses):.4f} | Val {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⏹️ Early stopping triggered")
            break

# restore best model
model.load_state_dict(best_state)

Epoch 1/50 | Train 0.6988 | Val 0.7011
Epoch 2/50 | Train 0.6966 | Val 0.7002
Epoch 3/50 | Train 0.6962 | Val 0.6993
Epoch 4/50 | Train 0.6949 | Val 0.6986
Epoch 5/50 | Train 0.6942 | Val 0.6979
Epoch 6/50 | Train 0.6932 | Val 0.6974
Epoch 7/50 | Train 0.6920 | Val 0.6970
Epoch 8/50 | Train 0.6918 | Val 0.6967
Epoch 9/50 | Train 0.6909 | Val 0.6965
Epoch 10/50 | Train 0.6901 | Val 0.6961
Epoch 11/50 | Train 0.6893 | Val 0.6960
Epoch 12/50 | Train 0.6882 | Val 0.6957
Epoch 13/50 | Train 0.6883 | Val 0.6954
Epoch 14/50 | Train 0.6886 | Val 0.6955
Epoch 15/50 | Train 0.6880 | Val 0.6954
Epoch 16/50 | Train 0.6892 | Val 0.6952
Epoch 17/50 | Train 0.6868 | Val 0.6950
Epoch 18/50 | Train 0.6873 | Val 0.6948
Epoch 19/50 | Train 0.6859 | Val 0.6948
Epoch 20/50 | Train 0.6862 | Val 0.6946
Epoch 21/50 | Train 0.6849 | Val 0.6946
Epoch 22/50 | Train 0.6843 | Val 0.6947
Epoch 23/50 | Train 0.6850 | Val 0.6946
Epoch 24/50 | Train 0.6834 | Val 0.6945
Epoch 25/50 | Train 0.6836 | Val 0.6947
Epoch 26/

<All keys matched successfully>

In [62]:
# Evaluation on test set
model.eval()
y_true_list, y_pred_list = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        preds = (outputs > 0.5).float()
        y_true_list.append(y_batch.numpy())
        y_pred_list.append(preds.numpy())

y_true = np.concatenate(y_true_list)
y_pred = np.concatenate(y_pred_list)

acc_per_yield = (y_true == y_pred).mean(axis=0)
overall_acc = (y_true == y_pred).mean()

print("\nAccuracy per yield:")
for name, acc in zip(target_cols, acc_per_yield):
    print(f"{name}: {acc:.3f}")
print(f"Overall accuracy: {overall_acc:.3f}")


Accuracy per yield:
Y_1MO: 0.519
Y_3MO: 0.552
Y_6MO: 0.519
Y_1year: 0.455
Y_2year: 0.461
Y_3year: 0.532
Y_5year: 0.474
Y_7year: 0.506
Y_10year: 0.481
Y_20year: 0.513
Y_30year: 0.519
Overall accuracy: 0.503
